# Notebook for XSEDE proposal

### Resource usage.

In [5]:
from yt import *
import numpy as np

Number of cells in the simulation.
Assume the zoom in leaves us 384 blocks in the inner zoomed region for ~50 pc^3 area,
which is what we get for the Bullhead cloud.

After that the volume filling fraction is about 1/8, so the number of blocks generally doubles per level of refinement. Here we use 2.5 to be safe (which was what Olson put in the original NSF proposal).

Also assume blocks have 16^3 cells in each.

In [6]:
lzoom=5
lmax=12

factor = 2.5

blksPerProc = 200

# Number of blocks for 50 pc zoom in region, based on the bull head cloud.
# These are all the background blocks that *don't* include the zoomed region.
numBlksOuter=3500

# Now the zoomed region at 5 levels of refinement and over a 50 pc region square
# should cover about 4^3=64 blocks. Then from there each level should just more than double the
# last level. But for some reason the first level in (lef=6) has 364 blocks, not
# 64*2.5=160 blocks. After that though it follows the 2.5 rule pretty well.

numBlksZoom1=384

numBlksLevel = numBlksZoom1

numBlksZoomTotal = numBlksZoom1

# Number of blocks if we assume a cloud that is an order magnitude larger has twice the
# radius and therefore 2^3 times more blocks. This is likely conservative, since the
# cloud is probably more dense and not as large.
numBlksBig=numBlksZoom1*(2^3)


# Each level has factor*numBlksPreviousLevel number of blocks.
# Here we sum up those blocks to get the total. Note we start
# with only those blocks in the region we zoomed in on!
# Previously we mistakingly calculated it with all the blocks
# at the previous level.
for i in range(1, lmax-lzoom+1):
    
    print i
    
    numBlksLevel = factor*numBlksLevel
     
    numBlksZoomTotal = numBlksZoomTotal + numBlksLevel
    
    print numBlksLevel
    
numBlksTotal = numBlksZoomTotal + numBlksOuter
    
print numBlksTotal, "blocks total."

print numBlksTotal/blksPerProc

numCells =  numBlksTotal*(16**3)

blksPerProc = 200

print numBlksTotal/blksPerProc

print numCells, "cells in the simulation after fragmentation."

1
960.0
2
2400.0
3
6000.0
4
15000.0
5
37500.0
6
93750.0
7
234375.0
393869.0 blocks total.
1969.345
1969.345
1613287424.0 cells in the simulation after fragmentation.


Here we calculate the average timestep assuming that the highest temperature is due to the radaition feedback. We assume it takes 2 Myr for the gas to collapse to the Jeans criterion at the highest level of refinement and star formation to start.

In [7]:
# Temp.
lowT = 10e4 * units.K
highT = 10e6 * units.K

# Sound speed.
csLow = np.sqrt(5./3. *units.boltzmann_constant_cgs.v*lowT.v /(2.4* units.mass_hydrogen_cgs.v))
csHigh = np.sqrt(5./3. *units.boltzmann_constant_cgs.v*highT.v /(2.4* units.mass_hydrogen_cgs.v))

print csLow, csHigh

# 500 AU cell size at highest refinement.
dx = 500 * units.AU.in_cgs().v

print dx

# Timestep.
lowTS = dx/csLow
highTS = dx/csHigh

print lowTS/(60*60*24*365.25), "years for low temp timesteps"
print highTS/(60*60*24*365.25), "years for high temp timesteps"


Myr = 3.15e13

#Myr = 15*Myr

# Number of steps for a 10 Myr simulation.
numSteps = 2*Myr/lowTS + 8*Myr/highTS

print numSteps, "steps in the simulation"



2393409.3641 23934093.641
7.47989353754e+15
99.031752283 years for low temp timesteps
9.9031752283 years for high temp timesteps
826505.933064 steps in the simulation


Now we calculate the amount of computer time needed to do the simulation. Here we assume:

1) An average Flash MHD update takes ~150 microseconds (Olson's timing).

2) An average radiation takes about the same amount of time as an MHD update (Wise & Abel 2011, Stone @ PiTP 2016)

In [8]:
timePerStep = 300e-6
#procSpeed = 3e9
hr = 60*60

print "{:.2e}".format(numSteps*numCells*timePerStep/hr), "hrs is total CPU time"

1.11e+08 hrs is total CPU time


### Storage requirements

Assume that we make a plot file every 10,000 years

In [23]:
pltFreq = 1e5
runTime = 1e7
numPlots = runTime/pltFreq

numHyVars = 30
numCells = numCells
numPartVars = 15
numParts = 1e4



sizeOfVar = 64
sizeOfGB = 2.**30

tempStorage = numPlots*(numHyVars*numCells+numPartVars*numParts)*sizeOfVar/sizeOfGB

print tempStorage

29744.2064881
